In [1]:
# stdlib
from random import randint
from typing import List
# third party
import numpy as np
import pytest

# syft absolute
from syft import deserialize
from syft import serialize
from syft.core.adp.entity import Entity
from syft.core.tensor.autodp.single_entity_phi import SingleEntityPhiTensor
from syft.core.tensor.autodp.row_entity_phi import RowEntityPhiTensor
from syft.core.tensor.tensor import Tensor

gonzalo = Entity(name="Gonzalo")


@pytest.fixture(scope="function")
def x() -> Tensor:
    x = Tensor(np.array([[1, 2, 3], [4, 5, 6]]))
    x = x.private(min_val=-1, max_val=7, entities=[gonzalo])
    return x


@pytest.fixture(scope="function")
def y() -> Tensor:
    y = Tensor(np.array([[-1, -2, -3], [-4, -5, -6]]))
    y = y.private(min_val=-7, max_val=1, entities=[gonzalo])
    return y


ent = Entity(name="test")
ent2 = Entity(name="test2")

dims = np.random.randint(10) + 1

child1 = np.random.randint(low=-2, high=4, size=dims)
upper1 = np.full(dims, 3, dtype=np.int32)
low1 = np.full(dims, -2, dtype=np.int32)
child2 = np.random.randint(low=4, high=7, size=dims)
upper2 = np.full(dims, 6, dtype=np.int32)
low2 = np.full(dims, 4, dtype=np.int32)

tensor1 = SingleEntityPhiTensor(
    child=child1, entity=ent, max_vals=upper1, min_vals=low1
)
# same entity, same data
tensor2 = SingleEntityPhiTensor(
    child=child1, entity=ent, max_vals=upper1, min_vals=low1
)
# same entity, different data
tensor3 = SingleEntityPhiTensor(
    child=child2, entity=ent, max_vals=upper2, min_vals=low2
)
# different entity, same data
tensor4 = SingleEntityPhiTensor(
    child=child1, entity=ent2, max_vals=upper1, min_vals=low1
)
# different entity, different data
tensor5 = SingleEntityPhiTensor(
    child=child2, entity=ent2, max_vals=upper2, min_vals=low2
)

simple_type1 = randint(-6, -4)
simple_type2 = randint(4, 6)


In [2]:
dims = np.random.randint(10) + 1
row_count = np.random.randint(10) + 1

def rept(low, high, entity) -> List:
    data = []
    for _ in range(row_count):
        data.append(
            SingleEntityPhiTensor(
                child = np.random.randint(low=low, high=high, size=dims),
                entity=entity,
                max_vals=np.full(dims, high-1, dtype=np.int32),
                min_vals=np.full(dims, low, dtype=np.int32)
            )
        )
    return RowEntityPhiTensor(rows=data, check_shape=False) 

tensor1 = rept(-2, 4, ent)
# same entity, same data
tensor2 = tensor1
# same entity, different data
tensor3 = rept(4, 7, ent)

simple_type1 = randint(-6, -4)
simple_type2 = randint(4, 6)


In [3]:
tensor1

RowEntityPhiTensor(child=[SingleEntityPhiTensor(entity=test, child=[2]), SingleEntityPhiTensor(entity=test, child=[3]), SingleEntityPhiTensor(entity=test, child=[-2]), SingleEntityPhiTensor(entity=test, child=[1]), SingleEntityPhiTensor(entity=test, child=[0]), SingleEntityPhiTensor(entity=test, child=[3])])

In [4]:
a_min = 1
a_max = 2
new_list = list()
for row in tensor1.child:
    new_list.append(row.clip(a_min=a_min, a_max=a_max))

In [5]:
a = RowEntityPhiTensor(rows=new_list)

In [6]:
for i in a.child:
    print((i.child >= 1).all())

True
True
True
True
True
True


In [7]:
rand1 = np.random.randint(-4, 1)
rand2 = np.random.randint(1, 5)
clipped_tensor1 = tensor1.clip(rand1, rand2).child
clipped_tensor2 = tensor1.clip(rand2, rand1).child
clipped_tensor3 = tensor1.clip(rand1, None).child


In [9]:
for i in clipped_tensor1:
    print((i.child >= rand1).all())

True
True
True
True
True
True
